<a href="https://colab.research.google.com/github/elbuendiego/MIP_Estatal/blob/main/MIP_Estatal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code for the "Strategic actions to promote a sustainable recovery Post-COVID" project within the ExpertODS initiative of the Sustainable Development Solutions Network (SDSN - Mexico).  

Author: Diego Alfaro

Instala librerias no incluidas en sesion Colab

In [87]:
!pip install xlsxwriter

     |████████████████████████████████| 149 kB 4.3 MB/s 


Importa librerias

In [88]:
import pandas as pd
import numpy as np
import xlsxwriter

Funcion auxiliar

In [89]:
def cols_index(df):
  '''
  Asigna sectores a columnas e indices de df

  input:
    df: pandas dataframe
  output:
    df: pandas datframe
  '''
  df.columns = sectores
  df.index = sectores
  return df

Lectura de archivo con MIPs estatales

In [28]:
xls = pd.ExcelFile('/content/MIP_Estatales_d_pb_ixi_2008.xlsx')

Analisis de sectores, iterando estado por estado y guardando un archivo xlsx por estado, incluyendo las hojas: 'MIP', 'COEFTEC', 'inv(I-A)', 'resultados'.

In [99]:
for n in range(1,33):
    num_estado = f'{n:02}'
    df = pd.read_excel(xls, f'E_{num_estado}') 
    nom_estado = df.iloc[2,13].split()[1:]
    nom_estado = '_'.join(nom_estado)
    sectores = df.iloc[6,6:25].values
    codigo = df.iloc[7,6:25].values

    prod_total = df.iloc[46,6:25]
    mip = df.iloc[9:28,6:25]
    coef_tec = mip / prod_total
    leont_mat = np.identity(19) - coef_tec.to_numpy(dtype='float')
    leont_mat = np.linalg.inv(leont_mat)

    mult_columna = np.sum(leont_mat, axis=0)
    ind_columna = mult_columna / np.mean(mult_columna)
    mult_fila =  np.sum(leont_mat, axis=1)
    ind_fila = mult_fila / np.mean(mult_fila)

    tipo_columna = np.where(ind_columna > 1,'Impulsor', '')
    tipo_fila = np.where(ind_fila > 1,'Expansor', '')

    tipo_sector = np.where((ind_fila>1) & (ind_columna>1),'clave','independiente')
    tipo_sector = np.where((ind_fila>1) & (ind_columna<=1),'inducido hacia adelante',tipo_sector)
    tipo_sector = np.where((ind_fila<=1) & (ind_columna>1),'expansor hacia atras',tipo_sector)

    resultados = pd.DataFrame(data = {'Sector': sectores,
                                      'Codigo': codigo,
                                      'Multiplicador columna': mult_columna,
                                      'indice Uj': ind_columna,
                                      'Uj > 1': tipo_columna,
                                      'Multiplicador fila': mult_fila,
                                      'Ui': ind_fila,
                                      'Ui > 1': tipo_fila,
                                      'Tipo Sector': tipo_sector},
                              index = np.arange(1,20))
    mip = cols_index(mip)
    coef_tec = cols_index(pd.DataFrame(data=coef_tec))
    leont_mat = cols_index(pd.DataFrame(data=leont_mat))

    writer = pd.ExcelWriter(f'AnalisisMIP_{nom_estado}.xlsx')
    mip.to_excel(writer, sheet_name='MIP')
    coef_tec.to_excel(writer, sheet_name='COEFTEC')
    leont_mat.to_excel(writer, sheet_name='inv(I-A)')
    resultados.to_excel(writer, sheet_name='resultados')
    writer.save()